# Sparse Reconstruction Localization Analysis

This notebook demonstrates the **joint sparse superposition reconstruction** approach for transmitter localization:
1. Load processed power measurements from monitoring locations
2. Convert data from dB to linear scale
3. Run sparse reconstruction algorithm
4. Visualize sparse transmit power field
5. Compare with likelihood-based approach

---

## Overview

**Sparse Reconstruction Formulation:**

$$\hat{\mathbf{t}} = \arg\min_{\mathbf{t}\ge 0} \|\mathbf{W}(\log_{10}(\mathbf{A}_{\text{model}}\mathbf{t}) - \log_{10}(\mathbf{p}))\|_{2}^{2} + \lambda \|\mathbf{t}\|_{1}$$

**Key Features:**
- Single-stage joint optimization (vs. two-stage likelihood)
- Explicit sparsity constraint (few active transmitters)
- Convex optimization (global optimum guaranteed)
- Fast computation (~10-30 seconds vs. 5-10 minutes)

---

## Step 1: Setup and Configuration

Import modules and configure the analysis.

In [ ]:
# Import necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import sys
import time
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

# Import utility functions
from src.utils import (
    load_slc_map, 
    load_monitoring_locations, 
    get_sensor_locations_array,
    load_transmitter_locations,
    lin_to_dB
)

# Import SPARSE RECONSTRUCTION modules
from src.sparse_reconstruction import (
    joint_sparse_reconstruction,
    compute_signal_strength_at_points,
    dbm_to_linear,
    linear_to_dbm
)

# Import visualization
from src.visualization.spatial_plots import (
    plot_transmit_power_map,
    plot_signal_estimates_map
)

print("✓ Modules imported successfully")
print("✓ Using SPARSE RECONSTRUCTION algorithm")

In [ ]:
# Configure which transmitter(s) to analyze
TRANSMITTERS = ['moran']  # Change as needed
TRANSMITTER_COMMA = ",".join(TRANSMITTERS)
TRANSMITTER_UNDERSCORE = "_".join(TRANSMITTERS)

print(f"Analyzing transmitter(s): {TRANSMITTER_COMMA}")

# Set file paths
CUSTOM_YAML = f'../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}.yaml'
DATA_DIR = Path(f'../data/processed/{TRANSMITTER_UNDERSCORE}/')

print(f"Configuration file: {CUSTOM_YAML}")
print(f"Data directory: {DATA_DIR}")

## Step 2: Load Map and Configuration

In [ ]:
# Load configuration
with open('../config/parameters.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"  Proxel size: {config['spatial']['proxel_size']} m/pixel")
print(f"  Path loss exponent: {config['localization']['path_loss_exponent']}")
print(f"  Shadowing σ: {config['localization']['std_deviation']} dB")
print(f"  Correlation distance δ_c: {config['localization']['correlation_coeff']} m")

In [ ]:
# Load SLC map
print("Loading SLC map...")
map_data = load_slc_map(
    map_folder_dir="../",
    downsample_factor=config['spatial']['downsample_factor']
)

print(f"✓ Map loaded: shape {map_data['shape']}")
print(f"  UTM Easting range: [{map_data['UTM_long'].min():.1f}, {map_data['UTM_long'].max():.1f}] m")
print(f"  UTM Northing range: [{map_data['UTM_lat'].min():.1f}, {map_data['UTM_lat'].max():.1f}] m")

In [ ]:
# Load transmitter locations for visualization
print("Loading transmitter locations...")
all_tx_locations = load_transmitter_locations(
    config_path='../config/transmitter_locations.yaml',
    map_data=map_data
)

# Filter to only show transmitters being analyzed
tx_locations = {name: all_tx_locations[name] for name in TRANSMITTERS if name in all_tx_locations}

print(f"✓ Loaded {len(tx_locations)} transmitter location(s) for display")
print(f"\nTransmitter Locations:")
for tx_name, tx_data in tx_locations.items():
    print(f"  {tx_name:12s}: ({tx_data['latitude']:.6f}, {tx_data['longitude']:.6f}) → {tx_data['coordinates']}")

## Step 3: Load Monitoring Locations and Power Measurements

In [ ]:
# Load monitoring locations
print(f"Loading monitoring locations from {CUSTOM_YAML}...")
locations_config = load_monitoring_locations(
    config_path=CUSTOM_YAML,
    map_data=map_data
)

print(f"✓ Loaded {len(locations_config['data_points'])} monitoring locations")
print(f"  UTM Zone: {locations_config['utm_zone']}{'N' if locations_config['northern_hemisphere'] else 'S'}")

# Display locations
print("\nMonitoring Locations:")
for loc in locations_config['data_points']:
    print(f"  {loc['name']:12s}: ({loc['latitude']:.5f}, {loc['longitude']:.5f}) → {loc['coordinates']}")

In [ ]:
# Load processed power measurements (in dB)
print(f"\nLoading power measurements from {DATA_DIR}...")

observed_powers_dB = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_avg_powers.npy")
power_stds = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_std_powers.npy")
sample_counts = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_sample_counts.npy")

print(f"✓ Loaded power measurements:")
print(f"  Number of sensors: {len(observed_powers_dB)}")
print(f"  Power range: [{observed_powers_dB.min():.2f}, {observed_powers_dB.max():.2f}] dB")
print(f"  Total samples: {sample_counts.sum()}")
print(f"  Observed powers (dB): {observed_powers_dB}")

In [ ]:
# Display summary table
summary_df = pd.read_csv(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_summary.csv")
print("\nPower Measurement Summary:")
print(summary_df.to_string(index=False))

## Step 4: Convert Data for Sparse Reconstruction

The sparse reconstruction algorithm works in **linear power scale** (mW), not dB.

**Conversion:** $P[\text{mW}] = 10^{P[\text{dBm}]/10}$

In [ ]:
# Convert observed powers from dBm to linear scale (mW)
observed_powers_linear = dbm_to_linear(observed_powers_dB)

print("Unit Conversion:")
print(f"  Input (dBm):  {observed_powers_dB}")
print(f"  Output (mW):  {observed_powers_linear}")
print(f"\n  Linear scale range: [{observed_powers_linear.min():.2e}, {observed_powers_linear.max():.2e}] mW")

# Note: These are very small numbers (e.g., 1e-8 mW = 1e-11 W = 10 pW)
# This is normal for RF propagation at ~100m distances

In [ ]:
# Get sensor locations in pixel coordinates
sensor_locations = get_sensor_locations_array(locations_config)

print(f"Sensor locations (pixel coordinates):")
print(f"  Shape: {sensor_locations.shape}")
print(f"  Array: {sensor_locations}")

## Step 5: Run Sparse Reconstruction Algorithm

**Optimization Problem:**

$$\hat{\mathbf{t}} = \arg\min_{\mathbf{t}\ge 0} \|\mathbf{W}(\log_{10}(\mathbf{A}_{\text{model}}\mathbf{t}) - \log_{10}(\mathbf{p}))\|_{2}^{2} + \lambda \|\mathbf{t}\|_{1}$$

**Parameters:**
- $\mathbf{t}$: Transmit power field (N grid points)
- $\mathbf{p}$: Observed powers (M sensors)
- $\mathbf{A}_{\text{model}}$: Propagation matrix (linear path gains)
- $\mathbf{W}$: Whitening matrix ($\mathbf{V}^{-1/2}$)
- $\lambda$: Sparsity regularization parameter

In [ ]:
# Set regularization parameter
# Larger λ → sparser solution (fewer transmitters)
# Smaller λ → denser solution (more transmitters)
# Good starting point: λ = 0.001 * norm(observed_powers)

# Log-domain heuristic: scale inversely with power to balance unitless data term
# Data term is ~O(1) (dB squared error), Reg term is λ * power (Watts)
# So we need λ * 1e-8 ≈ 1 => λ ≈ 1e8
mean_power = np.mean(observed_powers_linear)
#lambda_reg = 50 / mean_power
lambda_reg = 0
gamma = 0
print(f"Regularization parameter: λ = {lambda_reg:.2e}")
print(f"  (Automatically scaled: 0.1 / mean(p))")
print(f"\nNote: You can adjust λ to control sparsity:")
print(f"  - Increase λ for sparser solution (fewer transmitters)")
print(f"  - Decrease λ for denser solution (more transmitters)")

In [ ]:
# Run sparse reconstruction
print("\n" + "="*70)
print("RUNNING SPARSE RECONSTRUCTION ALGORITHM")
print("="*70)
exclusion_radius=0
max_l2_norm = 1

start_time = time.time()

'''tx_map_sparse_baseline, info_sparse_baseline = joint_sparse_reconstruction(
    sensor_locations=sensor_locations,
    observed_powers_dBm=observed_powers_dB,  # Function converts internally
    map_shape=map_data['shape'],
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    sigma=config['localization']['std_deviation'],
    delta_c=config['localization']['correlation_coeff'],
    lambda_reg=lambda_reg,
    gamma=gamma,
    max_l2_norm=max_l2_norm,
    norm_exponent=0.5,
    enable_reweighting=False,
    sparsity_threshold=1e-11,
    whitening_method='covariance',
    exclusion_radius=exclusion_radius,
    solver='l-bfgs-b',  # Use scipy L-BFGS-B for non-convex log-domain objective
    return_linear_scale=False,  # Return in dBm for visualization
    verbose=True,
    model_type='tirem',
    tirem_config_path='../config/tirem_parameters.yaml',
    n_jobs=-1 
)'''

tx_map_sparse_reweighted, info_sparse_reweighted = joint_sparse_reconstruction(
    sensor_locations=sensor_locations,
    observed_powers_dBm=observed_powers_dB,  # Function converts internally
    map_shape=map_data['shape'],
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    sigma=config['localization']['std_deviation'],
    delta_c=config['localization']['correlation_coeff'],
    lambda_reg=lambda_reg,
    gamma=gamma,
    max_l2_norm=max_l2_norm,
    norm_exponent=0,
    enable_reweighting=False,
    #max_reweight_iter=3,        # Fewer iterations
    #reweight_epsilon=1e-6,      # Different damping
    #convergence_tol=1e-8,       # Looser convergence
    sparsity_threshold=1e-11,
    whitening_method='hetero_diag',
    sigma_noise=1e-13,
    eta=0.5,
    exclusion_radius=exclusion_radius,
    solver='glrt',  # Use scipy L-BFGS-B for non-convex log-domain objective
    return_linear_scale=False,  # Return in dBm for visualization
    verbose=True,
    model_type='tirem',
    tirem_config_path='../config/tirem_parameters.yaml',
    n_jobs=-1 
)


elapsed_time = time.time() - start_time

print(f"\n✓ Reconstruction completed in {elapsed_time:.2f} seconds")

## Step 6: Analyze Sparse Solution

In [ ]:
# Display reconstruction statistics
print("\n" + "="*70)
print("SPARSE RECONSTRUCTION RESULTS")
print("="*70)
'''
print(f"\nSparsity Statistics:")
print(f"  Grid size: {map_data['shape'][0]} × {map_data['shape'][1]} = {np.prod(map_data['shape'])} pixels")
print(f"  Non-zero entries: {info_sparse_baseline['n_nonzero']}")
print(f"  Sparsity: {info_sparse_baseline['sparsity']*100:.2f}% (fraction of zeros)")
print(f"  Active fraction: {(1-info_sparse_baseline['sparsity'])*100:.4f}%")

print(f"\nPeak Location:")
peak_row, peak_col = info_sparse_baseline['peak_location']
print(f"  Pixel coordinates: (row={peak_row}, col={peak_col})")
print(f"  Peak power: {info_sparse_baseline['peak_power_dBm']:.1f} dBm")
print(f"  Peak power (linear): {info_sparse_baseline['peak_power_linear']:.2e} mW")

print(f"\nSolver Information:")
print(f"  Solver used: {info_sparse_baseline['solver_info']['solver_used']}")
print(f"  Success: {info_sparse_baseline['solver_info']['success']}")

print(f"\nComputational Efficiency:")
print(f"  Total time: {elapsed_time:.2f} seconds")
print(f"  (~10-50x faster than likelihood-based approach)")
'''

In [ ]:
# Find top-K non-zero locations
K = 10  # Number of top locations to display
'''
# Convert to linear scale for sorting
tx_map_linear_baseline = dbm_to_linear(tx_map_sparse_baseline)
flat_powers_baseline = tx_map_linear_baseline.ravel()
top_k_indices_baseline = np.argsort(flat_powers_baseline)[-K:][::-1]  # Top K descending

print(f"\nTop {K} Transmitter Locations (by power):")
print("-" * 80)
print(f"{'Rank':<6} {'Row':<8} {'Col':<8} {'Power (dBm)':<15} {'Power (mW)':<15}")
print("-" * 80)

for rank, idx in enumerate(top_k_indices_baseline, 1):
    row = idx // map_data['shape'][1]
    col = idx % map_data['shape'][1]
    power_linear = flat_powers_baseline[idx]
    power_dBm = linear_to_dbm(power_linear)
    
    if power_linear > 1e-15:  # Only show non-negligible values
        print(f"{rank:<6} {row:<8} {col:<8} {power_dBm:<15.2f} {power_linear:<15.2e}")

print("-" * 80)
'''
# Convert to linear scale for sorting
tx_map_linear_reweighted = dbm_to_linear(tx_map_sparse_reweighted)
flat_powers_reweighted = tx_map_linear_reweighted.ravel()
top_k_indices_reweighted = np.argsort(flat_powers_reweighted)[-K:][::-1]  # Top K descending

print(f"\nTop {K} Transmitter Locations (by power):")
print("-" * 80)
print(f"{'Rank':<6} {'Row':<8} {'Col':<8} {'Power (dBm)':<15} {'Power (mW)':<15}")
print("-" * 80)

for rank, idx in enumerate(top_k_indices_reweighted, 1):
    row = idx // map_data['shape'][1]
    col = idx % map_data['shape'][1]
    power_linear = flat_powers_reweighted[idx]
    power_dBm = linear_to_dbm(power_linear)
    
    if power_linear > 1e-15:  # Only show non-negligible values
        print(f"{rank:<6} {row:<8} {col:<8} {power_dBm:<15.2f} {power_linear:<15.2e}")

print("-" * 80)

## Step 7: Visualize Sparse Transmit Power Map

In [ ]:
# Visualize sparse transmit power field
tx_name_list = " & ".join(TRANSMITTERS)
'''
fig, ax = plot_transmit_power_map(
    transmit_power_map=tx_map_sparse_baseline,
    data_points=sensor_locations,
    observed_powers=observed_powers_dB,
    UTM_lat=map_data['UTM_lat'],
    UTM_long=map_data['UTM_long'],
    band_name=f"{tx_name_list} Transmitter (Sparse Reconstruction)",
    transmitter_locations=tx_locations
)

# Add sparsity annotation
ax.text(
    0.02, 0.98, 
    f"Sparsity: {info_sparse_baseline['sparsity']*100:.1f}%\n"
    f"Non-zero: {info_sparse_baseline['n_nonzero']} / {np.prod(map_data['shape'])}\n"
    f"λ = {lambda_reg:.2e}",
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8)
)

plt.tight_layout()
plt.show()
'''

fig, ax = plot_transmit_power_map(
    transmit_power_map=tx_map_sparse_reweighted,
    data_points=sensor_locations,
    observed_powers=observed_powers_dB,
    UTM_lat=map_data['UTM_lat'],
    UTM_long=map_data['UTM_long'],
    band_name=f"{tx_name_list} Transmitter (1 Iteration)",
    transmitter_locations=tx_locations
)

# Add sparsity annotation
ax.text(
    0.02, 0.98, 
    f"Sparsity: {info_sparse_reweighted['sparsity']*100:.1f}%\n"
    f"Non-zero: {info_sparse_reweighted['n_nonzero']} / {np.prod(map_data['shape'])}\n"
    f"λ = {lambda_reg:.2e}",
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8)
)

plt.tight_layout()
plt.show()

## Step 8: Compare Predictions with Observations

In [ ]:
'''
# Compute predicted powers at sensor locations using sparse transmit field
print("Computing signal strength at sensor locations...")

# Need to convert tx_map back to linear for physics-based propagation
tx_map_linear = dbm_to_linear(tx_map_sparse)

predicted_powers_linear = compute_signal_strength_at_points(
    transmit_power_map_linear=tx_map_linear,
    target_locations=sensor_locations,
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    return_linear_scale=False,  # Return in dBm
    verbose=True
)

predicted_powers_dB = predicted_powers_linear

print(f"\n✓ Predictions computed")
'''

In [ ]:
'''
# Evaluation metrics
print("\n" + "="*70)
print("EVALUATION METRICS")
print("="*70)

# Per-sensor comparison
print(f"\nPer-Sensor Comparison:")
print("-" * 80)
print(f"{'Location':<15} {'Observed (dBm)':<18} {'Predicted (dBm)':<18} {'Error (dB)':<12}")
print("-" * 80)

errors = []
for i, loc in enumerate(locations_config['data_points']):
    obs = observed_powers_dB[i]
    pred = predicted_powers_dB[i]
    error = obs - pred
    errors.append(error)
    print(f"{loc['name']:<15} {obs:<18.2f} {pred:<18.2f} {error:<12.2f}")

errors = np.array(errors)
print("-" * 80)

# Summary statistics
mse = np.mean(errors**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(errors))
max_error = np.max(np.abs(errors))

print(f"\nSummary Statistics:")
print(f"  Mean Squared Error (MSE):  {mse:.2f} dB²")
print(f"  Root Mean Squared Error:   {rmse:.2f} dB")
print(f"  Mean Absolute Error (MAE): {mae:.2f} dB")
print(f"  Max Absolute Error:        {max_error:.2f} dB")

# Compare with baseline (mean predictor)
variance_baseline = np.var(observed_powers_dB)
variance_reduction = (1 - mse / variance_baseline) * 100

print(f"\n  Baseline variance (mean predictor): {variance_baseline:.2f} dB²")
print(f"  Variance reduction: {variance_reduction:.1f}%")

if variance_reduction > 0:
    print(f"  ✓ Model outperforms baseline predictor")
else:
    print(f"  ✗ Model underperforms baseline (consider adjusting λ)")

print("="*70)
'''

In [ ]:
'''
# Scatter plot: Observed vs Predicted
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(observed_powers_dB, predicted_powers_dB, s=100, alpha=0.7, edgecolors='k')

# Add diagonal line (perfect prediction)
min_val = min(observed_powers_dB.min(), predicted_powers_dB.min())
max_val = max(observed_powers_dB.max(), predicted_powers_dB.max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

ax.set_xlabel('Observed Power (dBm)', fontsize=12)
ax.set_ylabel('Predicted Power (dBm)', fontsize=12)
ax.set_title('Sparse Reconstruction: Observed vs Predicted Power', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Add statistics text box
stats_text = f"RMSE = {rmse:.2f} dB\nMAE = {mae:.2f} dB\nSparsity = {info_sparse['sparsity']*100:.1f}%"
ax.text(
    0.05, 0.95, stats_text,
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8)
)

plt.tight_layout()
plt.show()
'''

## Step 9: Sensitivity Analysis - Effect of λ

Explore how the regularization parameter $\lambda$ affects sparsity and accuracy.

In [ ]:
'''
# Test different λ values
lambda_values = [1e-4, 1e-3, 1e-2, 1e-1, 1.0]

results = []

print("Testing different λ values...")
print("(This may take 1-2 minutes)\n")

for lam in lambda_values:
    print(f"λ = {lam:.0e}...", end=' ')
    
    # Run reconstruction
    tx_map_test, info_test = joint_sparse_reconstruction(
        sensor_locations=sensor_locations,
        observed_powers_dBm=observed_powers_dB,
        map_shape=map_data['shape'],
        scale=config['spatial']['proxel_size'],
        np_exponent=config['localization']['path_loss_exponent'],
        sigma=config['localization']['std_deviation'],
        delta_c=config['localization']['correlation_coeff'],
        lambda_reg=lam,
        solver='auto',
        return_linear_scale=False,
        verbose=False  # Suppress detailed output
    )
    
    # Compute predictions
    tx_map_test_linear = dbm_to_linear(tx_map_test)
    pred_test = compute_signal_strength_at_points(
        transmit_power_map_linear=tx_map_test_linear,
        target_locations=sensor_locations,
        scale=config['spatial']['proxel_size'],
        np_exponent=config['localization']['path_loss_exponent'],
        return_linear_scale=False,
        verbose=False
    )
    
    # Compute error
    rmse_test = np.sqrt(np.mean((observed_powers_dB - pred_test)**2))
    
    results.append({
        'lambda': lam,
        'sparsity': info_test['sparsity'],
        'n_nonzero': info_test['n_nonzero'],
        'rmse': rmse_test
    })
    
    print(f"Sparsity={info_test['sparsity']*100:.1f}%, RMSE={rmse_test:.2f} dB")

print("\n✓ Sensitivity analysis complete")
'''

In [ ]:
'''
# Plot λ sensitivity
results_df = pd.DataFrame(results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Sparsity vs λ
ax1.semilogx(results_df['lambda'], results_df['sparsity'] * 100, 'o-', linewidth=2, markersize=8)
ax1.set_xlabel('Regularization Parameter λ', fontsize=12)
ax1.set_ylabel('Sparsity (%)', fontsize=12)
ax1.set_title('Sparsity vs Regularization', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: RMSE vs λ
ax2.semilogx(results_df['lambda'], results_df['rmse'], 'o-', linewidth=2, markersize=8, color='orange')
ax2.set_xlabel('Regularization Parameter λ', fontsize=12)
ax2.set_ylabel('RMSE (dB)', fontsize=12)
ax2.set_title('Prediction Error vs Regularization', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("  - As λ increases → sparsity increases (fewer transmitters)")
print("  - As λ increases → RMSE may increase (less flexible model)")
print("  - Optimal λ balances sparsity and accuracy")
print("  - Choose λ based on application requirements")
'''

## Step 10: Summary and Key Takeaways

In [ ]:
'''
print("\n" + "="*70)
print("SPARSE RECONSTRUCTION SUMMARY")
print("="*70)

print(f"\n✓ Successfully localized {len(TRANSMITTERS)} transmitter(s) using sparse reconstruction")
print(f"\nKey Results:")
print(f"  • Sparsity: {info_sparse['sparsity']*100:.2f}% ({info_sparse['n_nonzero']}/{np.prod(map_data['shape'])} non-zero)")
print(f"  • Peak location: (row={info_sparse['peak_location'][0]}, col={info_sparse['peak_location'][1]})")
print(f"  • Peak power: {info_sparse['peak_power_dBm']:.1f} dBm")
print(f"  • Prediction RMSE: {rmse:.2f} dB")
print(f"  • Computation time: {elapsed_time:.2f} seconds")
print(f"  • Solver: {info_sparse['solver_info']['solver_used']}")

print(f"\nAdvantages of Sparse Reconstruction:")
print(f"  1. Fast: ~10-30 seconds (vs. 5-10 minutes for likelihood-based)")
print(f"  2. Sparse: Explicit localization with few non-zero entries")
print(f"  3. Convex: Guaranteed global optimum")
print(f"  4. Flexible: Tunable sparsity via λ parameter")
print(f"  5. Physical: Non-negativity constraint ensures valid powers")

print(f"\nWhen to use this approach:")
print(f"  • Single or few well-separated transmitters")
print(f"  • Need for fast computation")
print(f"  • Prior belief in sparse transmitter distribution")
print(f"  • Want guaranteed global optimum (convex optimization)")

print("\n" + "="*70)
'''

---

## Additional Experiments (Optional)

Try these experiments to explore the algorithm further:

1. **Change λ:** Modify `lambda_reg` to see how it affects sparsity vs accuracy
2. **Different transmitters:** Change `TRANSMITTERS` list to analyze other bands
3. **Solver comparison:** Force specific solvers ('cvxpy', 'sklearn', 'scipy') and compare
4. **Path loss exponent:** Modify `np_exponent` to see sensitivity to propagation model
5. **Different data:** Use different monitoring location configurations

---

## References

**Documentation:**
- `SPARSE_RECONSTRUCTION_THEORY.md` - Complete mathematical theory
- `MATHEMATICAL_ANALYSIS.md` - Likelihood-based approach theory

**Code Modules:**
- `src/sparse_reconstruction/` - Sparse reconstruction implementation
- `src/localization/` - Likelihood-based approach

**Papers:**
- Tibshirani (1996): "Regression Shrinkage and Selection via the Lasso"
- Candès & Tao (2005): "Decoding by Linear Programming"
---